<a href="https://colab.research.google.com/github/Arpit4477/p22io201/blob/main/SDE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#google_drive_link1 = '/content/drive/MyDrive/ColabDataSet/coco.names'
#google_drive_link2 = '/content/drive/MyDrive/ColabDataSet/yolov3.cfg'
#google_drive_link3 = '/content/drive/MyDrive/ColabDataSet/yolov3.weights'
import cv2
import numpy as np


# Load YOLO
net = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/ColabDataSet/yolov3.cfg", "/content/drive/MyDrive/ColabDataSet/yolov3.weights")
classes = []
with open("/content/drive/MyDrive/ColabDataSet/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayers()


# Capturing video
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")


while True:
    _, frame = cap.read()
    if frame is None:
        print("Error: Could not read frame.")
        break

        height, width, channels = frame.shape


    # Extract the right 33% of the frame
    right_frame = frame[:, int(2 * width / 3):]


    # Detect objects using YOLO on the right side
    blob = cv2.dnn.blobFromImage(right_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    outs = net.forward(output_layers_names)


    # Get information from the detection
    class_ids = []
    confidences = []
    boxes = []


    # Process each output layer
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Adjust confidence threshold as needed
                # Object detected
                center_x = int(detection[0] * (width / 3)) + int(2 * width / 3)  # Adjust x-coordinate for the full frame
                center_y = int(detection[1] * height)
                w = int(detection[2] * (width / 3))
                h = int(detection[3] * height)


                # Get the coordinates of the object
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)


                # Store information for displaying bounding box and text
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)


    # Apply non-maximum suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


    # Display names and bounding boxes of the detected objects
    for i in range(len(boxes)):
        if i in indices:
            x, y, w, h = boxes[i]


            # Draw bounding box and name
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


            # Apply perspective transformation based on the detected object's corners (optional)
            imgPts = np.float32([[x, y], [x + w, y], [x, y + h], [x + w, y + h]])
            objPoints = np.float32([[0, 0], [420, 0], [0, 637], [420, 637]])
            matrix = cv2.getPerspectiveTransform(imgPts, objPoints)
            result = cv2.warpPerspective(frame, matrix, (400, 600))


            # Display the original and corrected frames
            cv2.imshow('Object Detection', frame)
            cv2.imshow('Perspective Transformation', result)


    # Continue capturing frames even if no objects are detected


    # Check for user exit (press 'Esc' key)
    key = cv2.waitKey(1)
    if key == 27:
        break


# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

Error: Could not open camera.
Error: Could not read frame.
